Utworzneie heatmapy na podstawie współrzędnych przystanów i linii, które obsługują

In [1]:
# import bibliotek
import pandas as pd
import folium
from folium import plugins

In [2]:
# plik z przystankami z danego dnia
FILE_NAME = 'przystanki_2022-10-22.pkl'

In [3]:
# wczytanie pliku i komwersja typów
df = pd.read_pickle(FILE_NAME, compression='zip')
df['szer_geo'] = df['szer_geo'].astype(float)
df['dlug_geo'] = df['dlug_geo'].astype(float)

In [4]:
# obliczenie ilości linii i ich normalizacja (utworzenie wag)
df['n_linii'] = df['linie'].apply(lambda x: len(x))
df['n_linii_norm'] = df['n_linii'] / df['n_linii'].max()

In [5]:
# utworzenie mapy folium (start w punkcie centralnym)
map = folium.Map(location=[df.szer_geo.mean(), df.dlug_geo.mean()], zoom_start=14, control_scale=True) 

# tworzenie markerów
for index, data in df.iterrows():
    folium.CircleMarker(
        [data['szer_geo'], data['dlug_geo']], # współrzędne geograficzne markera
        popup=f'<b>{data.nazwa_zespolu} {data.slupek}</b><br>{data.linie}', # popup z nazwą zespołu, numerem słupka oraz spisem linii
        radius=2).add_to(map) # wielkość markera

# wydzielona tabela ze współrzędnymi geograficznymi oraz utworzonymi wcześniej wagami
lines_df = df[['szer_geo', 'dlug_geo', 'n_linii_norm']]

# dodanie heatmapy do już utworzonej mapy folium 
plugins.HeatMap(lines_df).add_to(map)

# zapis do pliku html
map.save('HEAT_MAP.HTML')